In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# read movie basics csv ignoring some columns
movieBasics = pd.read_csv('./input/movieBasics.csv', usecols=['tconst', 'isAdult', 'startYear', 'runtimeMinutes', 'genres'])
movieBasics.head()

,tconst,isAdult,startYear,runtimeMinutes,genres
0,tt0000009,0,1894,45,Romance
1,tt0000147,0,1897,20,"Documentary,News,Sport"
2,tt0000335,0,1900,\N,"Biography,Drama"
3,tt0000502,0,1905,100,\N
4,tt0000574,0,1906,70,"Biography,Crime,Drama"


In [3]:
# set up ratings table
ratings = pd.read_csv('./input/title.ratings.tsv', sep="\t")
ratings.sort_values(by=['numVotes'], ascending=False)

,tconst,averageRating,numVotes
81166,tt0111161,9.3,2270157
244017,tt0468569,9.0,2233191
553453,tt1375666,8.8,1991458
97366,tt0137523,8.8,1801220
80950,tt0110912,8.9,1774104
...,...,...,...
621085,tt1756261,8.4,5
186699,tt0327571,6.4,5
896960,tt5693174,9.0,5
505366,tt11861042,6.6,5


In [4]:
# set up crew table
crew = pd.read_csv('./input/title.crew.tsv', sep='\t')
crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [5]:
# set up the principles table
prin = pd.read_csv('./input/titlePrinciples.tsv', sep='\t', usecols=['tconst', 'nconst', 'category'])
prin.head()

,tconst,nconst,category
0,tt0000001,nm1588970,self
1,tt0000001,nm0005690,director
2,tt0000001,nm0374658,cinematographer
3,tt0000002,nm0721526,director
4,tt0000002,nm1335271,composer


In [6]:
# condense title principles into a smaller table that only has actors
actorPrin = prin[prin.category == 'actor']
actorPrin = actorPrin.drop(['category'], axis=1)
actorPrin.head()

,tconst,nconst
11,tt0000005,nm0443482
12,tt0000005,nm0653042
16,tt0000007,nm0179163
17,tt0000007,nm0183947
21,tt0000008,nm0653028


In [7]:
# put multiple actors on one line
actors = actorPrin.groupby('tconst')['nconst'].apply(','.join).reset_index()
actors.head()

,tconst,nconst
0,tt0000005,"nm0443482,nm0653042"
1,tt0000007,"nm0179163,nm0183947"
2,tt0000008,nm0653028
3,tt0000009,"nm0183823,nm1309758"
4,tt0000011,nm3692297


In [8]:
df1 = pd.merge(movieBasics,ratings,on='tconst',how='left')
df1.head()

,tconst,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,0,1894,45,Romance,5.9,154.0
1,tt0000147,0,1897,20,"Documentary,News,Sport",5.2,355.0
2,tt0000335,0,1900,\N,"Biography,Drama",6.1,41.0
3,tt0000502,0,1905,100,\N,3.8,6.0
4,tt0000574,0,1906,70,"Biography,Crime,Drama",6.1,588.0


In [9]:
df2 = pd.merge(df1,crew,on='tconst',how='left')
df2.head()

,tconst,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
0,tt0000009,0,1894,45,Romance,5.9,154.0,nm0085156,nm0085156
1,tt0000147,0,1897,20,"Documentary,News,Sport",5.2,355.0,nm0714557,\N
2,tt0000335,0,1900,\N,"Biography,Drama",6.1,41.0,"nm0095714,nm0675140",\N
3,tt0000502,0,1905,100,\N,3.8,6.0,nm0063413,"nm0063413,nm0657268,nm0675388"
4,tt0000574,0,1906,70,"Biography,Crime,Drama",6.1,588.0,nm0846879,nm0846879


In [10]:
df3 = pd.merge(df2,actors,on='tconst',how='left')
df3.head()
print(df3.shape)

(557550, 10)


In [11]:
# remove adult films
df4 = df3[df3.isAdult == 0]
print(df4.shape)

(548731, 10)


In [12]:
df5 = df4.drop(['tconst', 'isAdult'], axis=1)
df5.head()

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
0,1894,45,Romance,5.9,154.0,nm0085156,nm0085156,"nm0183823,nm1309758"
1,1897,20,"Documentary,News,Sport",5.2,355.0,nm0714557,\N,NaN
2,1900,\N,"Biography,Drama",6.1,41.0,"nm0095714,nm0675140",\N,"nm1012612,nm1011210,nm1012621,nm0675239,nm0675260"
3,1905,100,\N,3.8,6.0,nm0063413,"nm0063413,nm0657268,nm0675388","nm0215752,nm0252720"
4,1906,70,"Biography,Crime,Drama",6.1,588.0,nm0846879,nm0846879,"nm0846894,nm3002376"


In [318]:
# remove films with less than 100 ratings
df6 = df5[df5.numVotes > 9999]
print(df6.shape)

(8638, 8)


In [319]:
df6.to_csv('OurData2.zip', index=False)

In [320]:
df6.head()

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
936,1915,195,"Drama,History,War",6.3,22122.0,nm0000428,"nm0228746,nm0000428,nm0940488",nm0910400
1937,1916,163,"Drama,History",7.8,13835.0,nm0000428,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640...","nm0366008,nm0877548"
4436,1920,76,"Fantasy,Horror,Mystery",8.1,55301.0,nm0927468,"nm0562346,nm0417917","nm0470328,nm0891998,nm0270415"
6047,1921,68,"Comedy,Drama,Family",8.3,108411.0,nm0000122,nm0000122,"nm0088471,nm0000122,nm0001067,nm0588033,nm0042..."
6807,1922,91,"Fantasy,History,Horror",7.7,11721.0,nm0159725,nm0159725,nm0159725


In [321]:
#convert array of directors into single number value representing each director
def firstName(string):
    return string[2:].split(',', 1)[0]

df6.directors = pd.to_numeric(df6.directors.apply(firstName))
df6.head()

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
936,1915,195,"Drama,History,War",6.3,22122.0,428.0,"nm0228746,nm0000428,nm0940488",nm0910400
1937,1916,163,"Drama,History",7.8,13835.0,428.0,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640...","nm0366008,nm0877548"
4436,1920,76,"Fantasy,Horror,Mystery",8.1,55301.0,927468.0,"nm0562346,nm0417917","nm0470328,nm0891998,nm0270415"
6047,1921,68,"Comedy,Drama,Family",8.3,108411.0,122.0,nm0000122,"nm0088471,nm0000122,nm0001067,nm0588033,nm0042..."
6807,1922,91,"Fantasy,History,Horror",7.7,11721.0,159725.0,nm0159725,nm0159725


In [182]:
# create a dictionary for genres relating to a number
i = 0

# assign a new key value map
def assignNewKey(str):
    if ',' in str:
        str = str.split(',')
    if isinstance(str, list):
        [assignNewKey(element) for element in str]

    elif str not in gd.keys():
        global i
        i = i + 1
        gd[str] = i
        

gd = dict()
gd[''] = 0

# iterate through every genre and create a hashmap for genres
df6.genres.apply(assignNewKey)

print(gd)


{'': 0, 'Drama': 1, 'History': 2, 'War': 3, 'Fantasy': 4, 'Horror': 5, 'Mystery': 6, 'Comedy': 7, 'Family': 8, 'Documentary': 9, 'Action': 10, 'Thriller': 11, 'Romance': 12, 'Adventure': 13, 'Sci-Fi': 14, 'Biography': 15, 'Music': 16, 'Musical': 17, 'Crime': 18, 'Film-Noir': 19, 'Sport': 20, 'Animation': 21, 'Western': 22, 'News': 23}


In [351]:
# generate a list of genre combinations
perms = []

for i in range (0,24):
    for j in range (0, 24):
        for k in range (0, 24):
            nums = [i,j,k]
            nums.sort()
            nums = [str(element) for element in nums]
            nums = ",".join(nums)
            perms.append(nums)
            

perms = list(dict.fromkeys(perms))
print(perms)

, '2,14,22', '2,14,23', '2,15,15', '2,15,16', '2,15,17', '2,15,18', '2,15,19', '2,15,20', '2,15,21', '2,15,22', '2,15,23', '2,16,16', '2,16,17', '2,16,18', '2,16,19', '2,16,20', '2,16,21', '2,16,22', '2,16,23', '2,17,17', '2,17,18', '2,17,19', '2,17,20', '2,17,21', '2,17,22', '2,17,23', '2,18,18', '2,18,19', '2,18,20', '2,18,21', '2,18,22', '2,18,23', '2,19,19', '2,19,20', '2,19,21', '2,19,22', '2,19,23', '2,20,20', '2,20,21', '2,20,22', '2,20,23', '2,21,21', '2,21,22', '2,21,23', '2,22,22', '2,22,23', '2,23,23', '3,3,3', '3,3,4', '3,3,5', '3,3,6', '3,3,7', '3,3,8', '3,3,9', '3,3,10', '3,3,11', '3,3,12', '3,3,13', '3,3,14', '3,3,15', '3,3,16', '3,3,17', '3,3,18', '3,3,19', '3,3,20', '3,3,21', '3,3,22', '3,3,23', '3,4,4', '3,4,5', '3,4,6', '3,4,7', '3,4,8', '3,4,9', '3,4,10', '3,4,11', '3,4,12', '3,4,13', '3,4,14', '3,4,15', '3,4,16', '3,4,17', '3,4,18', '3,4,19', '3,4,20', '3,4,21', '3,4,22', '3,4,23', '3,5,5', '3,5,6', '3,5,7', '3,5,8', '3,5,9', '3,5,10', '3,5,11', '3,5,12', '3,5,13',

In [357]:
# getGenreCombo - usage, comma separated string of genres as input, returns combo numberz
myGenres1 = 'Action,Comedy'
myGenres2 = 'Comedy,Action'

def genresToNumbers(glist):
    glist = glist.split(',')
    glist = [gd[element] for element in glist]
    return glist

def genreNumsToComboNumAdd0(numList):
    if(len(numList) < 3):
        numList.append(0)
        genreNumsToComboNumAdd0(numList)
    return numList

def genreNumsToComboNum(numList):
    numList = genreNumsToComboNumAdd0(numList)
    numList.sort()
    numList = [str(element) for element in numList]
    numList = ",".join(numList)
    return perms.index(numList)

def getGenreCombo(genres):
    return genreNumsToComboNum(genresToNumbers(genres))

[2, 4, 23]


638

In [324]:
df6.genres = df6.genres.apply(getGenreCombo)
df6.head()

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
936,1915,195,324,6.3,22122.0,428.0,"nm0228746,nm0000428,nm0940488",nm0910400
1937,1916,163,25,7.8,13835.0,428.0,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640...","nm0366008,nm0877548"
4436,1920,76,1081,8.1,55301.0,927468.0,"nm0562346,nm0417917","nm0470328,nm0891998,nm0270415"
6047,1921,68,424,8.3,108411.0,122.0,nm0000122,"nm0088471,nm0000122,nm0001067,nm0588033,nm0042..."
6807,1922,91,620,7.7,11721.0,159725.0,nm0159725,nm0159725


In [325]:
df6.head()

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
936,1915,195,324,6.3,22122.0,428.0,"nm0228746,nm0000428,nm0940488",nm0910400
1937,1916,163,25,7.8,13835.0,428.0,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640...","nm0366008,nm0877548"
4436,1920,76,1081,8.1,55301.0,927468.0,"nm0562346,nm0417917","nm0470328,nm0891998,nm0270415"
6047,1921,68,424,8.3,108411.0,122.0,nm0000122,"nm0088471,nm0000122,nm0001067,nm0588033,nm0042..."
6807,1922,91,620,7.7,11721.0,159725.0,nm0159725,nm0159725


In [326]:
dfpop = df6.sort_values(['averageRating'], ascending=False)
dfpop

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
144278,1975,87,30,9.3,36163.0,251027.0,"nm0119457,nm0407642","nm0839017,nm0654805,nm0015145,nm0015081"
77820,1994,142,1,9.3,2270157.0,1104.0,"nm0000175,nm0001104","nm0000209,nm0000151,nm0348409,nm0006669"
368745,2013,139,149,9.2,43342.0,2339782.0,nm0948000,NaN
50222,1972,175,41,9.2,1565883.0,338.0,"nm0701374,nm0000338","nm0000008,nm0000199,nm0001001"
522753,2020,101,428,9.1,93621.0,2535507.0,"nm4264671,nm7929999,nm2354099,nm2352210,nm1981261","nm3818286,nm1306410,nm1384413"
...,...,...,...,...,...,...,...,...
321381,2011,105,187,1.6,75066.0,160840.0,\N,NaN
414120,2014,79,148,1.4,14788.0,229544.0,"nm0229544,nm6805281","nm0131647,nm0229544,nm3955339"
471077,2017,108,38,1.4,72142.0,1760911.0,nm8890606,"nm2900622,nm1055050,nm1513231,nm5589612,nm1758900"
426903,2015,114,141,1.3,27075.0,2930027.0,\N,"nm0876570,nm1203229,nm1127784,nm0477978,nm2127..."


In [335]:
dfpop = pd.merge(df1,crew,on='tconst',how='left')
dfpop.head()

,tconst,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
0,tt0000009,0,1894,45,Romance,5.9,154.0,nm0085156,nm0085156
1,tt0000147,0,1897,20,"Documentary,News,Sport",5.2,355.0,nm0714557,\N
2,tt0000335,0,1900,\N,"Biography,Drama",6.1,41.0,"nm0095714,nm0675140",\N
3,tt0000502,0,1905,100,\N,3.8,6.0,nm0063413,"nm0063413,nm0657268,nm0675388"
4,tt0000574,0,1906,70,"Biography,Crime,Drama",6.1,588.0,nm0846879,nm0846879


In [354]:
dfpop.directors = dfpop.directors.apply(firstName)

TypeError: 'NoneType' object is not subscriptable

In [356]:
getGenreCombo('History,Fantasy,News')z

638

In [360]:
df6.sort_values(by=['numVotes']) 

,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
80440,1996,101,148,5.3,10006.0,262693.0,nm0445552,"nm0005435,nm0682302,nm0101711,nm0124079"
11394,1928,76,424,8.1,10007.0,781292.0,"nm0115669,nm0513858,nm0267868,nm0090213,nm0604526","nm0000036,nm0329467,nm0102718"
23273,1941,116,35,8.0,10008.0,943758.0,"nm0375484,nm0462111,nm0662213,nm0132180","nm0003339,nm0137999"
37830,1958,93,352,7.3,10009.0,936404.0,"nm0310775,nm0063459","nm0000022,nm0000044,nm0912001,nm0223290"
530850,2019,85,428,6.1,10009.0,53228.0,"nm0229800,nm0229801","nm0000507,nm1283879,nm7619737"
...,...,...,...,...,...,...,...,...
77657,1994,154,41,8.9,1774104.0,233.0,"nm0000233,nm0000812","nm0000237,nm0000168,nm0000246"
91880,1999,139,1,8.8,1801220.0,399.0,"nm0657333,nm0880243","nm0000093,nm0001570,nm0001533,nm0340260"
299289,2010,148,2080,8.8,1991458.0,634240.0,nm0634240,"nm0000138,nm0330687,nm0913822"
210613,2008,152,479,9.0,2233191.0,634240.0,"nm0634300,nm0634240,nm0333060,nm0004170","nm0000288,nm0005132,nm0001173,nm0000323"
